# Deployment To Herkou
# READ-3

![Dir TREE](https://github.com/cd-x/plant-detect/tree.png)

this is our app.py

here WSGIServer is the one which will keep our server running 

werkzeug utilities provides functionality for file uploading and storing it temporarily

In [ ]:
from flask import Flask, render_template, request,url_for,redirect,send_from_directory
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer
import os




#part-prediction (moved from prediction.py)

from tensorflow.keras.models import load_model
import os
import json
import numpy as np
import cv2

import itertools
import random
from collections import Counter
from glob import iglob
import uuid
import base64

Loading our model

In [ ]:


MODEL_PATH = './model/acc9275own.h5'

model=load_model(MODEL_PATH)


getting 38 classes in a list so that we can retrive it from index

In [ ]:

with open('categories.json', 'r') as f:
    cat_to_name = json.load(f)
    classes = list(cat_to_name.values())
    
#print (classes)

loading and predicting we've talked in READ-2

In [ ]:


IMAGE_SIZE=(224,224)





def load_image(filename):
    img = cv2.imread(filename)
    #img = cv2.imread(os.path.join(image_dir, filename)) #<-- use in case of test through existing validation dataset
    img = cv2.resize(img, (IMAGE_SIZE[0], IMAGE_SIZE[1]) )
    img = img /255
    
    return img


def predict(image):
    probabilities = model.predict(np.asarray([image]))[0]
    class_idx = np.argmax(probabilities)
    
    return {classes[class_idx]: probabilities[class_idx]}


# def say_hello():
#     print('function added')
#     print(classes)
#     print(model.summary())

my_random_sring just encrypts the given filename

In [ ]:



#part-predicction
def get_as_base64(url):
    return base64.b64encode(requests.get(url).content)

def my_random_string(string_length=10):
    """Returns a random string of length string_length."""
    random = str(uuid.uuid4()) # Convert UUID format to a Python string.
    random = random.upper() # Make all characters uppercase.
    random = random.replace("-","") # Remove the UUID '-'.
    return random[0:string_length] # Return the random string.



Routing API request

In [ ]:


app = Flask(__name__)

root_dir = os.path.dirname(__file__)

ALLOWED_EXTENSIONS = set(['jpg', 'jpeg', 'png'])
app.config['UPLOAD_FOLDER']='uploads'

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1] in ALLOWED_EXTENSIONS


@app.route("/",methods=['GET'])
def index():
	return render_template('base.html',label='',imagesource='../uploads/example.jpg')


@app.route('/',methods=['GET','POST'])
def upload():
	if request.method == 'POST':
		file = request.files['file']
		#saving file to uploads directory
		result='please upload a file'
		if file and allowed_file(file.filename):

			filename=secure_filename(file.filename)
			file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
			file.save(file_path)

			#prediction part
			img = load_image(file_path)
			result = predict(img)
			filename= my_random_string(6) + filename
			os.rename(file_path, os.path.join(app.config['UPLOAD_FOLDER'], filename))
	return render_template('base.html',imagesource='../uploads/'+filename,label=result)


#upload API
@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'],
                               filename)


**This part is more important since heroku doesn't provide any image storing memory it just gives ram so to process the image we need to store the image somewhere below code does it very cleverly it srores the data temporarily in the ram and as soon as its done it removes the file from memory**

```
from werkzeug.middleware.shared_data import SharedDataMiddleware
```
Making SharedDataMiddleWare's build_only parameter True will do the same 

In [ ]:



from werkzeug.middleware.shared_data import SharedDataMiddleware
app.add_url_rule('/uploads/<filename>', 'uploaded_file',
                 build_only=True)
app.wsgi_app = SharedDataMiddleware(app.wsgi_app, {
    '/uploads':  app.config['UPLOAD_FOLDER']
})

if __name__ == '__main__':
	app.run(debug=False, threaded=False)

# heroku deployment process



1.   If project works fine on localhost make Debug=False

2.   $pip freeze > requirements.txt

2.   $git init

3.   $git add .

4.   $git commit -m "deploying"

5.   $git remote add origin [ git-repo-url ]

6.   $git push -u origin master

8.   create new app on heroku 

9.   connect with your repo using option connect with github

10.  $git push heroku master

**Few important commands**

$heroku logs --tail --app [app-name]

In case of memory quota exceeded ERROR

set WEB_CONCURRENCY TO 1 OR 2

In case if you want to make any changes to your running project 

1.   add change locally to your git repository 

2.  perform git staging

3.  run command '10' mentioned above heroku itself will look for change if there is any in requirements.txt it will reinstall ooonly that item or if there is only change in code it will just change that code and will start re-building. It takes just few seconds once done voila !! you have your new version of the app.

**!! Stay alert if slug size is more than 500 mb your app will crash !!**

